In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
sys.path.append('/usr0/home/naveenr/projects/patient_provider')

In [5]:
import numpy as np
import random 
import matplotlib.pyplot as plt


In [127]:
from patient.simulator import Simulator
from patient.policy import *

In [155]:
num_patients = 20
num_providers = 10

In [156]:
s = Simulator(num_patients,num_providers)

In [157]:
seed_list = list(range(42,52))

In [158]:
results = s.simulate_no_renetry(random_policy,seed_list=seed_list)
np.mean(results), np.std(results)

(14.5, 1.284523257866513)

In [159]:
results = s.simulate_no_renetry(greedy_policy,seed_list=seed_list)
np.mean(results), np.std(results)

(16.6, 0.66332495807108)

In [160]:
results = s.simulate_no_renetry(brute_force_discount,seed_list=seed_list)
np.mean(results), np.std(results)

(16.3, 0.9)

In [161]:
s = Simulator(num_patients,num_providers)
results = s.simulate_with_renetry(random_policy,seed_list=seed_list)
np.mean(results['matches']), np.std(results['matches']), np.mean(results['waittimes']), np.std(results['waittimes'])

(16.4, 0.9165151389911679, 0.17, 0.44844174649557333)

In [162]:
s = Simulator(num_patients,num_providers)
results = s.simulate_with_renetry(greedy_policy,seed_list=seed_list)
np.mean(results['matches']), np.std(results['matches']), np.mean(results['waittimes']), np.std(results['waittimes'])

(16.7, 1.5524174696260022, 0.1, 0.33166247903553997)

In [163]:
results = s.simulate_with_renetry(brute_force_discount,seed_list=seed_list)
np.mean(results['matches']), np.std(results['matches']), np.mean(results['waittimes']), np.std(results['waittimes'])

(17.0, 1.5491933384829668, 0.105, 0.33760183648789593)

In [164]:
results = s.simulate_with_renetry(brute_force_discount_lamb,seed_list=seed_list)
np.mean(results['matches']), np.std(results['matches']), np.mean(results['waittimes']), np.std(results['waittimes'])

(17.3, 1.5524174696260025, 0.09, 0.31921779399024736)